# 推論用コード
Hugging Faceにアップロードしたモデルを用いてELYZA-tasks-100-TVの出力を得るためのコードです。  

In [1]:
# llama-cpp-pythonをCUDAサポート付きでインストール
!pip uninstall -y llama-cpp-python
!CMAKE_ARGS="-DGGML_CUDA=on -DCUDA_PATH=/usr/local/cuda-12.2 -DCUDAToolkit_ROOT=/usr/local/cuda-12.2 -DCUDAToolkit_INCLUDE_DIR=/usr/local/cuda-12/include -DCUDAToolkit_LIBRARY_DIR=/usr/local/cuda-12.2/lib64 -DCMAKE_CUDA_COMPILER:PATH=/usr/local/cuda/bin/nvcc" FORCE_CMAKE=1 pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122 --upgrade --force-reinstall --no-cache-dir
!pip install -U ipywidgets
!pip install transformers==4.46.3
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U datasets
!pip install -U peft==0.13.2
!pip install janome

In [16]:
import json
import logging
from llama_cpp import Llama
from tqdm import tqdm
from typing import Dict, Any
import numpy as np
import logging
from dataclasses import dataclass
import re
from janome.tokenizer import Tokenizer

In [ ]:
# ロギングの設定
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [4]:
# モデル設定
# GGUFモデルのパス
MODEL_PATH = "./llm-jp-3-13b-q8_0.gguf"
# LoRAアダプタのパス
lora_path = "llm-jp-3-13b-SFT-LoRA.gguf"
FILE_NAME = "llm-jp-3-13b-SFT-LoRA-Q8_0.jsonl"

In [ ]:
INSTRUCTIONS = {
    "内容を確認し、適切な対応をしてください。指示が不明確な場合は、具体的にどの部分が不明確なのかを質問し、指示の明確化を求めてください。回答は、簡潔かつ明確に記述してください。"
}

In [ ]:
FEW_SHOT_EXAMPLES = [
    {
        "input": "次の文は1つの物語の文の順番をシャッフルしたものです。適切な順に並び替えてください。\n\n1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」\n2. 老人は、広い畑で色々な果物を育てていました。\n3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。\n4. 昔、あるところに老人と孫がいました。\n5. 老人は微笑みながら次のように答えました。\n",
        "output": "物語の冒頭から順番に考えます。\n\n最初の文として適切なのは状況の開示を行っている4で、その後2が続きます。\nその後3で会話が始まり、会話への応答を表す5、1と続くと考えられます。\n\nよって適切な順番は4, 2, 3, 5, 1です。"
    },
    {
        "input": "「彼はとても頭が良い」という文を、別の言い方で表現してください。",
        "output": "「彼はとても頭が良い」という文は、以下のように言い換えることができます。\n\n*   彼は非常に**聡明**だ。\n*   彼は**頭脳明晰**だ。\n*   彼は**抜群に頭の回転が速い**。\n*   彼は**並外れた知性の持ち主**だ。\n*   彼は**才気煥発**だ。\n\nこれらの表現は、いずれも「頭が良い」という意味をより強調したり、具体的に表現したりしています。"
    },
    {
        "input": "次の文章を英語に翻訳してください。\n\n今日は良い天気ですね。",
        "output": "The weather is nice today, isn't it? **(直訳: 今日は天気が良いですね、そう思いませんか？)**\n\n**その他の表現:**\n\n*   It's a beautiful day today. (今日は美しい日ですね。)\n*   It's a lovely day, isn't it? (今日は素晴らしい日ですね、そう思いませんか？)\n*   We're having a nice day today. (今日は良い日ですね。)"
    },
    {
        "input": "次の果物を五十音順に並び替えてください。\n\nりんご、バナナ、みかん、いちご、ぶどう",
        "output": "いちご、ぶどう、バナナ、みかん、りんご\n\n**並び替えのルール:** 果物の名前を五十音順に並べ替えます。最初の文字が同じ場合は、次の文字で比較します。これを、単語の終わりまで繰り返します。"
    },
    {
        "input": "以下の表から、すべての科目名を抽出してください。\n| 科目名｜講師名｜教室名\n| 微積分学101｜田中教授｜202号室\n| 美術史｜山田（代理）｜ホールA",
        "output": "表には以下の2つの科目名が示されています:\n\n- 微積分学101\n- 美術史"
    },
    {
        "input": "今日の夕方、東京タワーに行くことはできる？\nという問いに答えるのに十分な情報はどれですか？\n\n情報1: 今日は3/1(金)で、明日は祝日だ。\n情報2: 東京タワーの営業状況について: 新型コロナウイルス感染拡大防止の為、営業時間を変更致します。 平日は21時まで、祝日は19時までの短縮営業となります。\n\n選択肢:\n- 情報1のみで十分で、情報2のみでは不十分\n- 情報2のみで十分で、情報1のみでは不十分\n- 情報1と2の両方で十分\n- 情報1, 2のどちらかのみで十分\n- 情報1, 2のどちらでも不十分",
        "output": "情報1で今日の日付について述べられていますが、情報2から東京タワーは平日・祝日問わず夕方の時間帯は営業していることがわかります。\n\nよって与えられた問いに答えるには「情報2のみで十分で、情報1のみでは不十分」です。"
    },
    {
        "input": "youtubeの台本を書くのを手伝ってください。\n\nタローとコタローという2人のキャラクターの対話ベースで進行します。\n\nタローは物知りで、コタローの疑問に答えてくれます。\n\n今回の動画は、映画スラムダンクを視聴者に薦める動画です。以下のポイントを押さえた脚本にしてください。\n\n- まるでスポーツの試合を見ているかのように迫力のあるバスケのシーン\n- 漫画の原作を読んだことのない人でも楽しめる、映画として完結しているストーリー\n- 来月に公開が終了する\n- 10行~15行程度の対話",
        "output": "わかりました。次のような台本はどうでしょうか？\n\nコタロー: いや〜次の土日何しようかな〜\nタロー: お、それならスラムダンクの映画を見るのはどう？\nコタロー: 映画見るのはいいね。タローはみたの？\nタロー: 見たよ！最高だったから是非見てほしいな。\nコタロー: へぇ。スラムダンクってバスケの漫画だっけ？\nタロー: そうそう。バスケのシーンはまるでスポーツの試合を見ているかのように迫力があるんだ！\nコタロー: 面白そうだね、だけどスラムダンクの原作読んだことないからな〜\nタロー: 原作を読んだことなくても、映画としてストーリーが完結しているから全然楽しめると思うよ！\nコタロー: そうなんだ、せっかくだから観に行こうかな\nタロー: 来月に公開が終了しちゃうから、ちょうど良かったね\nコタロー: そうなんだ！危ない危ない。来週なら間に合うから良かった。\nタロー: じゃあ、観たら感想教えてよね！"
    },
    {
        "input": "あなたは、小説の執筆者です。以下は小説の一文です。\n「風が強く吹いていた。」\nこの文章に対して、どのような表現を追加することで、風の強さがより強く表現できるか、答えを書いてください。",
        "output": "以下のような表現はどうでしょうか？\n\n- 「嵐のような強い風が吹き荒れていた。」\n- 「風の勢いはますます強くなり、とっくに飛ばされた帽子が道路を転がるように流されていた。」\n- 「風が強く吹きすさんで、まるで私たちを吹き飛ばそうとしているかのようだった。」\n- 「風が荒れ模様な中、落ち葉が舞い上がり、その音が響き渡っていた。」"
    },
    {
        "input": "1を3で割った結果に33を足して、210を掛ける。最後に100で割る。",
        "output": "ステップバイステップに数式を完成させていきます。\n\n1. 「1を3で割った結果に33を足す」は 1/3+33 とできます\n2. これに210をかけるには、直前が足し算なので全体を括弧で囲う必要があり、 (1/3+33)*210 になります\n3. 最後に100で割るには、直前が掛け算なのでそのまま (1/3+33)*210/100 となります\n\nよって元の数式を四則演算の数式として表すと (1/3+33)*210/100 になります。"
    },
    {
        "input": "「犬」と「猫」の違いを3つ挙げてください。",
        "output": "「犬」と「猫」の違いを3つ挙げると、以下のようになります。\n\n| 項目 | 犬 | 猫 |\n|---|---|---| \n| **分類** | 哺乳綱食肉目イヌ科イヌ属 | 哺乳綱食肉目ネコ科ネコ属 |\n| **性格** | 社交的で、飼い主に従順な傾向がある | 独立心が強く、気ままな性格の傾向がある |\n| **食性** | 雑食性（肉、魚、野菜など何でも食べる） | 肉食性（肉や魚を好む） |\n\nこれらの違いは、犬と猫の進化の過程や、人間との関わりの歴史の違いに由来しています。犬は古くから人間のパートナーとして、狩猟や牧畜などを手伝ってきました。そのため、人間とのコミュニケーション能力が高く、飼い主に従順な性格に進化したと考えられます。一方、猫は単独で狩りをする動物であり、独立心が強い性格をしています。近年では、猫もペットとして人気が高まっており、人間との関わり方も変化してきています。"
    }
]

In [ ]:
PARAMS = {'max_tokens': 1024, 'temperature': 0.2, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.1}

In [ ]:
def create_prompt(input_text):
    """タスク種別に応じたプロンプトの生成"""
    if not input_text:
        return "エラー: 入力テキストが空です。", "エラー", {}

    instruction = INSTRUCTIONS

    # Few-shotの例を追加
    few_shot_examples = ""
    for example in FEW_SHOT_EXAMPLES:
        few_shot_examples += f"### 入力:\n{example['input']}\n\n### 回答:\n{example['output']}\n\n===\n\n"

    prompt = f"### 指示:\n以下の指示に従って、『簡潔』で分かりやすい回答を心がけて回答してください。\n{instruction}\n\n{few_shot_examples}### 入力:\n{input_text}\n\n### 回答:\n"

    return prompt, PARAMS

In [ ]:
def create_prompts(data_list):
    """データセット全体に対してプロンプトを生成"""
    prompts = []
    params_list = []
    for data in data_list:
        input_text = data['input']
        prompt, params = create_prompt(input_text)
        prompts.append(prompt)
        params_list.append(params)
    return prompts, params_list

In [ ]:
# グローバル変数としてTokenizerを一度だけ初期化
tokenizer = Tokenizer()

def validate_output(output: str) -> bool:
    """出力のバリデーションを行う関数"""
    if not basic_validation(output):
        return False
    if not quality_validation(output):
        return False
    if not instruction_validation(output):
        return False
    if not detect_unnecessary_phrases(output):
        return False
    return True

def basic_validation(text: str) -> bool:
    """基本的なバリデーションチェック"""
    if not text or text.isspace():
        print("出力が空またはスペースのみです")
        return False
    
    if len(text) < 10:
        print("出力が短すぎます")
        return False
    if len(text) > 2000:
        print("出力が長すぎます")
        return False
    
    if has_excessive_repetition(text):
        print("過度な文字列の繰り返しが検出されました")
        return False
    
    return True

def quality_validation(text: str) -> bool:
    """文章の品質チェック"""
    if not validate_readability(text):
        print("読みやすさに問題があります")
        return False
    
    return True

def instruction_validation(text: str) -> bool:
    """指示に沿った出力がされているかチェック"""
    if "### 回答:" in text:
        print("出力に'### 回答:'が含まれています")
        return False
    
    if "### 入力:" in text:
        print("出力に'### 入力:'が含まれています")
        return False
    
    return True

def detect_unnecessary_phrases(text: str) -> bool:
    """不要な文言が含まれていないかチェックする関数"""
    unnecessary_phrases = [
        r"再確認:.*",
        r"###",
        r"入力:.*",
        r"回答:.*",
        r"応答:.*",
        r"応答例:.*",
        r"評価基準:.*",
        r"指示:.*",
        r"68文字.*",
        r"指示が不明瞭な場合は.*"
        r"指示が不明確な場合は.*"
        r"最後にもう一度.*"
    ]
    for phrase in unnecessary_phrases:
        if re.search(phrase, text, re.DOTALL):
            print(f"不要な文言「{phrase}」が検出されました")
            return False
    return True

def has_excessive_repetition(text: str) -> bool:
    """過度な繰り返しのチェック（最適化版）"""
    # 文字数の閾値を事前に計算
    text_len = len(text)
    char_threshold = text_len * 0.3
    
    # 文字の出現回数をカウント（頻出文字のみ）
    char_count = {}
    for char in text:
        # すでに閾値を超えている文字があれば即座にTrue
        if char in char_count:
            char_count[char] += 1
            if char_count[char] > char_threshold:
                return True
        else:
            char_count[char] = 1
    
    # 単語の繰り返しチェックは、明らかな繰り返しパターンがある場合のみ実行
    if len(set(text)) / text_len < 0.2:  # 文字の種類が少ない場合のみ
        words = [token.surface for token in tokenizer.tokenize(text)]
        words_len = len(words)
        word_threshold = words_len * 0.2
        
        word_count = {}
        for word in words:
            if len(word) > 1:  # 2文字以上の単語のみチェック
                if word in word_count:
                    word_count[word] += 1
                    if word_count[word] > word_threshold:
                        return True
                else:
                    word_count[word] = 1
    
    return False

def validate_readability(text: str) -> bool:
    """読みやすさのチェック"""
    # 文の長さチェック
    sentences = re.split('[。！？!?]', text)
    sentences = [s for s in sentences if s.strip()]
    
    for sent in sentences:
        if len(sent) > 150:
            return False
    
    # 漢字の割合チェック
    kanji_ratio = len([c for c in text if 0x4E00 <= ord(c) <= 0x9FFF]) / len(text)
    if kanji_ratio > 0.8:
        return False
    
    return True


In [ ]:
@dataclass
class ParamConfig:
    """パラメータ設定のデータクラス"""
    base_temp: float
    base_top_p: float
    base_top_k: int
    base_repeat_penalty: float
    
    def get_decay_rate(self, retry_count: int) -> float:
        """リトライ回数に応じた減衰率を計算"""
        return np.exp(-0.05 * retry_count) # 減衰率の計算式を変更

def adjust_generation_params(
    params: Dict[str, Any], 
    retry_count: int, 
    success_history: list = None
) -> Dict[str, Any]:
    """リトライ回数に応じて生成パラメータを最適化する関数"""
    
    # 基本設定
    config = ParamConfig(
        base_temp=0.7,
        base_top_p=0.9,
        base_top_k=40,
        base_repeat_penalty=1.3
    )
    
    # 元のパラメータをコピー
    adjusted_params = params.copy()
    
    # 成功率に基づく調整パラメータの計算
    if success_history:
        success_rate = sum(success_history) / len(success_history)
        # 成功率を-1から1の範囲にマッピング
        mu = (success_rate - 0.5) * 2
        sigma = 1.0
    else:
        mu = 0.0
        sigma = 1.0
        
    # リトライ回数に応じた基本調整
    decay_rate = config.get_decay_rate(retry_count)
    
    if retry_count == 1:
        # 1回目: 簡潔さと明瞭さを重視したベースライン設定
        adjusted_params.update({
            "temperature": config.base_temp,
            "top_p": config.base_top_p,
            "top_k": config.base_top_k,
            "repeat_penalty": config.base_repeat_penalty
        })
    elif retry_count == 2:
        # 2回目: 
        # バランス重視
        adjusted_params.update({
            "temperature": config.base_temp,
            "top_p": config.base_top_p,
            "top_k": config.base_top_k,
            "repeat_penalty": config.base_repeat_penalty
        })
    elif retry_count == 3:
        # 3回目: 成功率に基づいた調整
        if success_history and success_rate < 0.5:
            # 成功率が低い場合は探索を広げる
            adjusted_params.update({
                "temperature": min(config.base_temp + 0.2, 1.0), # 探索範囲を広げる
                "top_p": min(config.base_top_p + 0.1, 1.0), # 探索範囲を広げる
                "top_k": min(config.base_top_k + 10, 60), # 探索範囲を広げる
                "repeat_penalty": max(config.base_repeat_penalty - 0.2, 1.0) # 探索範囲を広げる
            })
        else:
            # 成功率が高い場合は保守的な設定
            adjusted_params.update({
                "temperature": max(config.base_temp - 0.1, 0.1),
                "top_p": max(config.base_top_p - 0.1, 0.5),
                "top_k": max(config.base_top_k - 10, 10),
                "repeat_penalty": min(config.base_repeat_penalty + 0.1, 2.5)
            })
    elif retry_count == 4:
        # 4回目: 減衰率に基づいた微調整
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.1 * decay_rate, 0.2), # 下限を緩和
            "top_p": max(config.base_top_p - 0.05 * decay_rate, 0.7), # 下限を緩和
            "top_k": max(int(config.base_top_k * (1 - 0.05 * decay_rate)), 30), # 下限を緩和
            "repeat_penalty": min(config.base_repeat_penalty + 0.1 * decay_rate, 1.8) # 範囲を調整
        })
    elif retry_count == 5:
        # 5回目: 減衰率に基づいた微調整
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.15 * decay_rate, 0.15), # 下限を緩和
            "top_p": max(config.base_top_p - 0.1 * decay_rate, 0.6), # 下限を緩和
            "top_k": max(int(config.base_top_k * (1 - 0.1 * decay_rate)), 25), # 下限を緩和
            "repeat_penalty": min(config.base_repeat_penalty + 0.15 * decay_rate, 2.0) # 範囲を調整
        })
    elif retry_count == 6:
        # 6回目: 減衰率に基づいた微調整
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.2 * decay_rate, 0.1), # 下限を緩和
            "top_p": max(config.base_top_p - 0.15 * decay_rate, 0.5), # 下限を緩和
            "top_k": max(int(config.base_top_k * (1 - 0.15 * decay_rate)), 20), # 下限を緩和
            "repeat_penalty": min(config.base_repeat_penalty + 0.2 * decay_rate, 2.2) # 範囲を調整
        })
    elif retry_count == 7:
        # 7回目: 減衰率に基づいた微調整
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.25 * decay_rate, 0.1), # 下限を緩和
            "top_p": max(config.base_top_p - 0.2 * decay_rate, 0.4), # 下限を緩和
            "top_k": max(int(config.base_top_k * (1 - 0.2 * decay_rate)), 15), # 下限を緩和
            "repeat_penalty": min(config.base_repeat_penalty + 0.25 * decay_rate, 2.5) # 範囲を調整
        })
    elif retry_count == 8:
        # 8回目: 最終調整
        # 保守的だが探索範囲を広げる
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.3 * decay_rate, 0.1), # 下限を緩和
            "top_p": max(config.base_top_p - 0.25 * decay_rate, 0.3), # 下限を緩和
            "top_k": max(int(config.base_top_k * (1 - 0.25 * decay_rate)), 10), # 下限を緩和
            "repeat_penalty": min(config.base_repeat_penalty + 0.3 * decay_rate, 2.8), # 範囲を調整
            "presence_penalty": 1.0,
            "frequency_penalty": 1.0
        })
    
    # パラメータの範囲を確認・調整
    adjusted_params["temperature"] = np.clip(adjusted_params["temperature"], 0.1, 1.0)
    adjusted_params["top_p"] = np.clip(adjusted_params["top_p"], 0.1, 1.0)
    adjusted_params["top_k"] = max(1, int(adjusted_params["top_k"]))
    
    logging.info(f"Adjusted parameters for retry {retry_count}: {adjusted_params}")
    
    return adjusted_params

In [ ]:
def generate_response(model, prompt, generation_params, task_id):
    max_retries = 15
    retries = 0
    success_history = []  # 成功履歴を追加

    while retries < max_retries:
        try:
            current_params = adjust_generation_params(
                generation_params, 
                retries + 1, 
                success_history
            )
            
            response = model.create_completion(
                prompt=prompt,
                **current_params
            )
            output = response["choices"][0]["text"].strip()

            if validate_output(output):
                success_history.append(1)
                return output
            else:
                success_history.append(0)
                logger.info(f"バリデーションエラー: タスクID {task_id}, リトライ {retries + 1} 回目")
                retries += 1

                # プロンプトに指示を追加
                if retries == 1:
                    # 簡潔で分かりやすい回答を強調
                    prompt += "\n\n**注意:** 回答は簡潔かつ要点を絞って記述してください。冗長さは避け、明確で理解しやすい説明を心がけてください。"
                elif retries == 2:
                    # 具体例やステップバイステップでの説明を必要に応じて追加
                    prompt += "\n\n**改善のためのヒント:** 必要であれば、具体的な例を挙げて説明してください。複雑な問題の場合は、ステップバイステップで段階的に説明すると分かりやすくなります。"
                elif retries == 3:
                    # 指示に厳密に従うように強調
                    prompt += "\n\n**再確認:** 最初の指示に厳密に従って回答してください。特に「簡潔」で「分かりやすい」回答を心がけてください。"
                elif retries == 4:
                    # 冗長さの回避を再度強調
                    prompt += "\n\n**警告:** 冗長さは評価を下げる大きな要因となります。簡潔さを最優先に、無駄な表現を徹底的に排除してください。"
                elif retries == 5:
                    # 具体例の使用を再度強調
                    prompt += "\n\n**具体例の確認:** 具体例を用いることで、説明が格段に分かりやすくなります。適切な具体例が使われているか、再度確認してください。"
                elif retries == 6:
                    # ステップバイステップの説明を再度強調
                    prompt += "\n\n**ステップバイステップの確認:** 複雑な問題の場合は、ステップバイステップで説明することで、理解が容易になります。手順が明確に示されているか、再度確認してください。"
                elif retries == 7:
                    # 指示への準拠を再度強調
                    prompt += "\n\n**指示の再確認:** これまでの指示をすべて満たしているか、再度確認してください。特に、簡潔さ、具体例の使用、ステップバイステップの説明に注意してください。"
                elif retries == 8:
                    # 最終確認と警告
                    prompt += "\n\n**最終確認:** これが最後のチャンスです。これまでの指示をすべて考慮し、最も適切で簡潔な回答を作成してください。不明瞭な点や冗長さがないか、最終確認を怠らないでください。"

        except Exception as e:
            logger.error(f"推論エラー: タスクID {task_id}, リトライ {retries + 1} 回目, エラー: {str(e)}")
            retries += 1

    logger.error(f"最大リトライ回数を超えました: タスクID {task_id}")
    return ''

In [ ]:
def main():
    """メイン処理"""
    # モデルの読み込み
    llm = Llama(
        model_path=MODEL_PATH,
        lora_path=lora_path,
        n_ctx=4096,          # コンテキストサイズを調整
        n_gpu_layers=-1,     # GPU層を増やす
        verbose=True         # 詳細なログを有効化
    )

    # テストデータの読み込み
    with open('elyza-tasks-100-TV_0.jsonl', 'r', encoding='utf-8') as f:
        test_data = [json.loads(line) for line in f]

    # プロンプトの事前生成
    prompts, params_list = create_prompts(test_data)

    print(prompts[0])

    # 推論処理
    results = []
    for prompt, params, data in tqdm(zip(prompts, params_list, test_data), desc="推論処理中", total=len(test_data)):
        input_text = data['input']
        task_id = data['task_id']

        # generate_response 関数を呼び出して推論を実行
        output = generate_response(llm, prompt, params, task_id)

        results.append({
            "task_id": task_id,
            'input': input_text,
            'output': output,
            'generation_params': {
                'max_tokens': params.get('max_tokens'),
                'temperature': params.get('temperature')
            }
        })

    # 結果の保存
    with open(FILE_NAME, 'w', encoding='utf-8') as f:
        for result in results:
            json.dump(result, f, ensure_ascii=False)
            f.write('\n')

In [ ]:
if __name__ == "__main__":
    main()